# Branch 2: 데이터 전처리 효과 비교

## 🎯 분석 목적
Branch 1에서 선정된 모델(XGBoost, TCN) 대상 전처리 효과 비교

## 📊 실험 구조

### **Branch 1 결과**
- **1순위**: XGBoost (RMSE: 72.29, Overfit Gap: 26.37)
- **2순위**: TCN (RMSE: 74.59, Overfit Gap: 21.08)

### **Branch 2 실험**
두 모델 모두에 대해 전처리 효과를 비교하여 최적 조합 도출

## 🔬 전처리 방법

### **Phase 1: 4가지 전처리 조합**
1. **Baseline**: 전처리 없음 (원본 데이터)
2. **RUL Clipping**: RUL 값을 125로 제한
3. **Noise Removal**: 낮은 상관도 센서 8개 제거
4. **Both**: RUL Clipping + Noise Removal

### **Phase 2: 상세 분석 (선택적)**
- RUL 임계값 탐색 (100, 125, 150, 175, 200)
- Noise 센서 개별 효과 분석
- 센서 상관계수 분석

## 📈 평가 지표
- **Test RMSE**: 테스트 셋 RMSE (낮을수록 좋음)
- **개선율**: Baseline 대비 개선 정도
- **Overfit Gap**: Train과 Test RMSE 차이 (작을수록 좋음)

In [1]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

print('✅ Google Drive 마운트 완료')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 마운트 완료


In [2]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, GlobalAveragePooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping

from scipy import stats

In [3]:
# 한글 폰트 설정
import matplotlib.font_manager as fm

font_path = '/content/drive/MyDrive/ML project/NanumGothic-Regular.ttf'
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)

plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

# 파일 경로 설정
DATA_PATH = '/content/drive/MyDrive/ML project'

# 전처리 파라미터
RUL_THRESHOLD = 125
NOISE_SENSORS = ['s_1', 's_5', 's_8', 's_14', 's_15', 's_16', 's_18', 's_19']
WINDOW_SIZE = 30

In [4]:
# 원본 데이터 로드
train_original = pd.read_csv(f'{DATA_PATH}/train_df.csv')
test_original = pd.read_csv(f'{DATA_PATH}/test_df.csv')

# 센서 컬럼
SENSOR_COLS = [col for col in train_original.columns if col.startswith('s_')]
SIGNAL_SENSORS = [s for s in SENSOR_COLS if s not in NOISE_SENSORS]

In [5]:
# 전처리 함수
def apply_rul_clipping(data, threshold):
    data_processed = data.copy()
    data_processed['RUL'] = data_processed['RUL'].clip(upper=threshold)
    return data_processed

def apply_noise_removal(data, noise_sensors):
    data_processed = data.copy()
    return data_processed

def create_sequences(data, sensors, window):
    X, y = [], []
    for unit in data['unit'].unique():
        unit_data = data[data['unit'] == unit].sort_values('cycle')
        values = unit_data[sensors].values
        targets = unit_data['RUL'].values

        for i in range(len(values) - window + 1):
            X.append(values[i:i+window])
            y.append(targets[i+window-1])

    return np.array(X), np.array(y)


## Section 1: XGBoost 전처리 비교

In [6]:
results_xgb = []

# XGBoost 모델 파라미터 (Branch 1과 동일)
xgb_params = {
    'n_estimators': 500,
    'max_depth': 7,
    'learning_rate': 0.05,
    'random_state': 42,
    'n_jobs': -1
}

In [7]:
# 1. XGBoost - Baseline
train_b = train_original.copy()
test_b = test_original.copy()

# StandardScaler
scaler = StandardScaler()
train_b_scaled = train_b.copy()
test_b_scaled = test_b.copy()
scaler.fit(train_b[SENSOR_COLS])
train_b_scaled[SENSOR_COLS] = scaler.transform(train_b[SENSOR_COLS])
test_b_scaled[SENSOR_COLS] = scaler.transform(test_b[SENSOR_COLS])

X_train = train_b_scaled[SENSOR_COLS].values
y_train = train_b_scaled['RUL'].values
X_test = test_b_scaled[SENSOR_COLS].values
y_test = test_b_scaled['RUL'].values

model_xgb = XGBRegressor(**xgb_params)
model_xgb.fit(X_train, y_train, verbose=False)

y_pred_train = model_xgb.predict(X_train)
y_pred_test = model_xgb.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse

results_xgb.append({
    'config': 'Baseline',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': 0.0
})

baseline_rmse_xgb = test_rmse

In [8]:
# 2. XGBoost - RUL Clipping
train_r = apply_rul_clipping(train_original, RUL_THRESHOLD)
test_r = apply_rul_clipping(test_original, RUL_THRESHOLD)

scaler = StandardScaler()
train_r_scaled = train_r.copy()
test_r_scaled = test_r.copy()
scaler.fit(train_r[SENSOR_COLS])
train_r_scaled[SENSOR_COLS] = scaler.transform(train_r[SENSOR_COLS])
test_r_scaled[SENSOR_COLS] = scaler.transform(test_r[SENSOR_COLS])

X_train = train_r_scaled[SENSOR_COLS].values
y_train = train_r_scaled['RUL'].values
X_test = test_r_scaled[SENSOR_COLS].values
y_test = test_r_scaled['RUL'].values

model_xgb = XGBRegressor(**xgb_params)
model_xgb.fit(X_train, y_train, verbose=False)

y_pred_train = model_xgb.predict(X_train)
y_pred_test = model_xgb.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse
improvement = ((baseline_rmse_xgb - test_rmse) / baseline_rmse_xgb) * 100

results_xgb.append({
    'config': 'RUL Clipping',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': improvement
})

In [9]:
# 3. XGBoost - Noise Removal
train_n = apply_noise_removal(train_original, NOISE_SENSORS)
test_n = apply_noise_removal(test_original, NOISE_SENSORS)

scaler = StandardScaler()
train_n_scaled = train_n.copy()
test_n_scaled = test_n.copy()
scaler.fit(train_n[SIGNAL_SENSORS])
train_n_scaled[SIGNAL_SENSORS] = scaler.transform(train_n[SIGNAL_SENSORS])
test_n_scaled[SIGNAL_SENSORS] = scaler.transform(test_n[SIGNAL_SENSORS])

X_train = train_n_scaled[SIGNAL_SENSORS].values
y_train = train_n_scaled['RUL'].values
X_test = test_n_scaled[SIGNAL_SENSORS].values
y_test = test_n_scaled['RUL'].values

model_xgb = XGBRegressor(**xgb_params)
model_xgb.fit(X_train, y_train, verbose=False)

y_pred_train = model_xgb.predict(X_train)
y_pred_test = model_xgb.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse
improvement = ((baseline_rmse_xgb - test_rmse) / baseline_rmse_xgb) * 100

results_xgb.append({
    'config': 'Noise Removal',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': improvement
})

In [10]:
# 4. XGBoost - Both
train_both = apply_noise_removal(apply_rul_clipping(train_original, RUL_THRESHOLD), NOISE_SENSORS)
test_both = apply_noise_removal(apply_rul_clipping(test_original, RUL_THRESHOLD), NOISE_SENSORS)

scaler = StandardScaler()
train_both_scaled = train_both.copy()
test_both_scaled = test_both.copy()
scaler.fit(train_both[SIGNAL_SENSORS])
train_both_scaled[SIGNAL_SENSORS] = scaler.transform(train_both[SIGNAL_SENSORS])
test_both_scaled[SIGNAL_SENSORS] = scaler.transform(test_both[SIGNAL_SENSORS])

X_train = train_both_scaled[SIGNAL_SENSORS].values
y_train = train_both_scaled['RUL'].values
X_test = test_both_scaled[SIGNAL_SENSORS].values
y_test = test_both_scaled['RUL'].values

model_xgb = XGBRegressor(**xgb_params)
model_xgb.fit(X_train, y_train, verbose=False)

y_pred_train = model_xgb.predict(X_train)
y_pred_test = model_xgb.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse
improvement = ((baseline_rmse_xgb - test_rmse) / baseline_rmse_xgb) * 100

results_xgb.append({
    'config': 'Both',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': improvement
})

In [11]:
# XGBoost 결과
df_xgb = pd.DataFrame(results_xgb)
df_xgb

,config,train_rmse,test_rmse,test_r2,overfit_gap,improvement
0,Baseline,45.922928,72.294111,0.384756,26.371183,0.000000
1,RUL Clipping,14.126525,16.234389,0.593538,2.107864,77.543967
2,Noise Removal,46.832134,72.875626,0.374818,26.043492,-0.804375
3,Both,14.633652,16.479174,0.581188,1.845522,77.205372



## Section 2: TCN 전처리 비교

Branch 1 2순위 모델인 TCN에 대한 전처리 효과 분석

In [12]:
results_tcn = []
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [13]:
# 1. TCN - Baseline
train_b = train_original.copy()
test_b = test_original.copy()

scaler = StandardScaler()
train_b_scaled = train_b.copy()
test_b_scaled = test_b.copy()
scaler.fit(train_b[SENSOR_COLS])
train_b_scaled[SENSOR_COLS] = scaler.transform(train_b[SENSOR_COLS])
test_b_scaled[SENSOR_COLS] = scaler.transform(test_b[SENSOR_COLS])

X_train, y_train = create_sequences(train_b_scaled, SENSOR_COLS, WINDOW_SIZE)
X_test, y_test = create_sequences(test_b_scaled, SENSOR_COLS, WINDOW_SIZE)

model_tcn = Sequential([
    Input(shape=(WINDOW_SIZE, len(SENSOR_COLS))),
    Conv1D(64, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    Conv1D(32, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

model_tcn.compile(optimizer='adam', loss='mse')
model_tcn.fit(X_train, y_train, epochs=50, batch_size=512,
             validation_split=0.2, callbacks=[early_stop], verbose=0)

y_pred_train = model_tcn.predict(X_train, verbose=0).flatten()
y_pred_test = model_tcn.predict(X_test, verbose=0).flatten()

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse

results_tcn.append({
    'config': 'Baseline',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': 0.0
})

baseline_rmse_tcn = test_rmse

In [14]:
# 2. TCN - RUL Clipping
train_r = apply_rul_clipping(train_original, RUL_THRESHOLD)
test_r = apply_rul_clipping(test_original, RUL_THRESHOLD)

scaler = StandardScaler()
train_r_scaled = train_r.copy()
test_r_scaled = test_r.copy()
scaler.fit(train_r[SENSOR_COLS])
train_r_scaled[SENSOR_COLS] = scaler.transform(train_r[SENSOR_COLS])
test_r_scaled[SENSOR_COLS] = scaler.transform(test_r[SENSOR_COLS])

X_train, y_train = create_sequences(train_r_scaled, SENSOR_COLS, WINDOW_SIZE)
X_test, y_test = create_sequences(test_r_scaled, SENSOR_COLS, WINDOW_SIZE)

model_tcn = Sequential([
    Input(shape=(WINDOW_SIZE, len(SENSOR_COLS))),
    Conv1D(64, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    Conv1D(32, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

model_tcn.compile(optimizer='adam', loss='mse')
model_tcn.fit(X_train, y_train, epochs=50, batch_size=512,
             validation_split=0.2, callbacks=[early_stop], verbose=0)

y_pred_train = model_tcn.predict(X_train, verbose=0).flatten()
y_pred_test = model_tcn.predict(X_test, verbose=0).flatten()

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse
improvement = ((baseline_rmse_tcn - test_rmse) / baseline_rmse_tcn) * 100

results_tcn.append({
    'config': 'RUL Clipping',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': improvement
})

In [15]:
# 3. TCN - Noise Removal
train_n = apply_noise_removal(train_original, NOISE_SENSORS)
test_n = apply_noise_removal(test_original, NOISE_SENSORS)

scaler = StandardScaler()
train_n_scaled = train_n.copy()
test_n_scaled = test_n.copy()
scaler.fit(train_n[SIGNAL_SENSORS])
train_n_scaled[SIGNAL_SENSORS] = scaler.transform(train_n[SIGNAL_SENSORS])
test_n_scaled[SIGNAL_SENSORS] = scaler.transform(test_n[SIGNAL_SENSORS])

X_train, y_train = create_sequences(train_n_scaled, SIGNAL_SENSORS, WINDOW_SIZE)
X_test, y_test = create_sequences(test_n_scaled, SIGNAL_SENSORS, WINDOW_SIZE)

model_tcn = Sequential([
    Input(shape=(WINDOW_SIZE, len(SIGNAL_SENSORS))),
    Conv1D(64, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    Conv1D(32, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

model_tcn.compile(optimizer='adam', loss='mse')
model_tcn.fit(X_train, y_train, epochs=50, batch_size=512,
             validation_split=0.2, callbacks=[early_stop], verbose=0)

y_pred_train = model_tcn.predict(X_train, verbose=0).flatten()
y_pred_test = model_tcn.predict(X_test, verbose=0).flatten()

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse
improvement = ((baseline_rmse_tcn - test_rmse) / baseline_rmse_tcn) * 100

results_tcn.append({
    'config': 'Noise Removal',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': improvement
})

In [16]:
# 4. TCN - Both
train_both = apply_noise_removal(apply_rul_clipping(train_original, RUL_THRESHOLD), NOISE_SENSORS)
test_both = apply_noise_removal(apply_rul_clipping(test_original, RUL_THRESHOLD), NOISE_SENSORS)

scaler = StandardScaler()
train_both_scaled = train_both.copy()
test_both_scaled = test_both.copy()
scaler.fit(train_both[SIGNAL_SENSORS])
train_both_scaled[SIGNAL_SENSORS] = scaler.transform(train_both[SIGNAL_SENSORS])
test_both_scaled[SIGNAL_SENSORS] = scaler.transform(test_both[SIGNAL_SENSORS])

X_train, y_train = create_sequences(train_both_scaled, SIGNAL_SENSORS, WINDOW_SIZE)
X_test, y_test = create_sequences(test_both_scaled, SIGNAL_SENSORS, WINDOW_SIZE)

model_tcn = Sequential([
    Input(shape=(WINDOW_SIZE, len(SIGNAL_SENSORS))),
    Conv1D(64, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    Conv1D(32, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

model_tcn.compile(optimizer='adam', loss='mse')
model_tcn.fit(X_train, y_train, epochs=50, batch_size=512,
             validation_split=0.2, callbacks=[early_stop], verbose=0)

y_pred_train = model_tcn.predict(X_train, verbose=0).flatten()
y_pred_test = model_tcn.predict(X_test, verbose=0).flatten()

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
overfit_gap = test_rmse - train_rmse
improvement = ((baseline_rmse_tcn - test_rmse) / baseline_rmse_tcn) * 100

results_tcn.append({
    'config': 'Both',
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'overfit_gap': overfit_gap,
    'improvement': improvement
})

In [17]:
# TCN 결과
df_tcn = pd.DataFrame(results_tcn)
df_tcn

,config,train_rmse,test_rmse,test_r2,overfit_gap,improvement
0,Baseline,53.507228,74.647698,0.298337,21.140470,0.000000
1,RUL Clipping,20.503852,20.114497,0.457664,-0.389355,73.054096
2,Noise Removal,55.188104,74.478578,0.301512,19.290474,0.226557
3,Both,21.521716,20.315982,0.446745,-1.205735,72.784182


## Section 3: XGBoost vs TCN 비교 분석

두 모델의 최적 전처리 조합 비교

In [18]:
# 최종 비교
best_xgb = df_xgb.loc[df_xgb['test_rmse'].idxmin()]
best_tcn = df_tcn.loc[df_tcn['test_rmse'].idxmin()]

comparison = [
    {
        'model': 'XGBoost',
        'best_config': best_xgb['config'],
        'test_rmse': best_xgb['test_rmse'],
        'overfit_gap': best_xgb['overfit_gap'],
        'improvement': best_xgb['improvement']
    },
    {
        'model': 'TCN',
        'best_config': best_tcn['config'],
        'test_rmse': best_tcn['test_rmse'],
        'overfit_gap': best_tcn['overfit_gap'],
        'improvement': best_tcn['improvement']
    }
]

df_comparison = pd.DataFrame(comparison)
df_comparison

,model,best_config,test_rmse,overfit_gap,improvement
0,XGBoost,RUL Clipping,16.234389,2.107864,77.543967
1,TCN,RUL Clipping,20.114497,-0.389355,73.054096


In [19]:
# CSV 저장
df_xgb.to_csv(f'{DATA_PATH}/branch2_xgboost_results.csv', index=False)
df_tcn.to_csv(f'{DATA_PATH}/branch2_tcn_results.csv', index=False)
df_comparison.to_csv(f'{DATA_PATH}/branch2_final_comparison.csv', index=False)